In [104]:
import os
from glob import glob

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from kitti_utils import *

## Load in Files

In [105]:
PREFIX = "../data"

DATE = "2011_09_26"
ID = "0005"

DATA_PATH = f"{PREFIX}/{DATE}/{DATE}_drive_{ID}_sync"

CAMERA_CALIB_FILE = f"{PREFIX}/{DATE}/calib_cam_to_cam.txt"

VELO_CALIB_FILE = f"{PREFIX}/{DATE}/calib_velo_to_cam.txt"

IMU_CALIB_FILE = f"{PREFIX}/{DATE}/calib_imu_to_velo.txt"

In [106]:
# get RGB camera data
left_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_02/data/*.png')))
right_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_03/data/*.png')))

# get LiDAR data
bin_paths = sorted(glob(os.path.join(DATA_PATH, 'velodyne_points/data/*.bin')))

# get GPS/IMU data
oxts_paths = sorted(glob(os.path.join(DATA_PATH, r'oxts/data**/*.txt')))

print(f"Number of left images: {len(left_image_paths)}")
print(f"Number of right images: {len(right_image_paths)}")
print(f"Number of LiDAR point clouds: {len(bin_paths)}")
print(f"Number of GPS/IMU frames: {len(oxts_paths)}")

Number of left images: 154
Number of right images: 154
Number of LiDAR point clouds: 154
Number of GPS/IMU frames: 154


In [107]:
with open(CAMERA_CALIB_FILE,'r') as f:
    calib = f.readlines()

## Extract Projection Matrices

In [108]:
def extract_rect_matrices(calib):
    rect_matrices = ""
    
    j = 0
    
    for i in range(9, len(calib), 8):
        P_rect = calib[i].strip().split(':')
        P_rect = f"P{j}: " + P_rect[1]

        rect_matrices += P_rect + "\n"

        j += 1
    
    return rect_matrices

In [109]:
rect_matrices = extract_rect_matrices(calib)
rect_matrices

'P0:  7.215377e+02 0.000000e+00 6.095593e+02 0.000000e+00 0.000000e+00 7.215377e+02 1.728540e+02 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00\nP1:  7.215377e+02 0.000000e+00 6.095593e+02 -3.875744e+02 0.000000e+00 7.215377e+02 1.728540e+02 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00\nP2:  7.215377e+02 0.000000e+00 6.095593e+02 4.485728e+01 0.000000e+00 7.215377e+02 1.728540e+02 2.163791e-01 0.000000e+00 0.000000e+00 1.000000e+00 2.745884e-03\nP3:  7.215377e+02 0.000000e+00 6.095593e+02 -3.395242e+02 0.000000e+00 7.215377e+02 1.728540e+02 2.199936e+00 0.000000e+00 0.000000e+00 1.000000e+00 2.729905e-03\n'

## Misc

In [110]:
# get projection matrices (rectified left camera --> left camera (u,v,z))
P_rect2_cam2 = np.array([float(x) for x in calib[25].strip().split(' ')[1:]]).reshape((3,4))


# get rectified rotation matrices (left camera --> rectified left camera)
R_ref0_rect2 = np.array([float(x) for x in calib[24].strip().split(' ')[1:]]).reshape((3, 3,))

# save the original
original_R_ref0_rect2 = R_ref0_rect2

# add (0,0,0) translation and convert to homogeneous coordinates
R_ref0_rect2 = np.insert(R_ref0_rect2, 3, values=[0,0,0], axis=0)
R_ref0_rect2 = np.insert(R_ref0_rect2, 3, values=[0,0,0,1], axis=1)


# get rigid transformation from Camera 0 (ref) to Camera 2
R_2 = np.array([float(x) for x in calib[21].strip().split(' ')[1:]]).reshape((3,3))
t_2 = np.array([float(x) for x in calib[22].strip().split(' ')[1:]]).reshape((3,1))


# get cam0 to cam2 rigid body transformation in homogeneous coordinates
T_ref0_ref2 = np.insert(np.hstack((R_2, t_2)), 3, values=[0,0,0,1], axis=0)     

In [111]:
T_velo_ref0 = get_rigid_transformation(VELO_CALIB_FILE)
T_imu_velo = get_rigid_transformation(IMU_CALIB_FILE)


# transform from velo (LiDAR) to left color camera (shape 3x4)
T_velo_cam2 = P_rect2_cam2 @ R_ref0_rect2 @ T_ref0_ref2 @ T_velo_ref0


# homogeneous transform from left color camera to velo (LiDAR) (shape: 4x4)
T_cam2_velo = np.linalg.inv(np.insert(T_velo_cam2, 3, values=[0,0,0,1], axis=0))


# transform from IMU to left color camera (shape 3x4)
T_imu_cam2 = T_velo_cam2 @ T_imu_velo


# homogeneous transform from left color camera to IMU (shape: 4x4)
T_cam2_imu = np.linalg.inv(np.insert(T_imu_cam2, 3, values=[0,0,0,1], axis=0)) 

## Extract Rectified Rotation Matrix

In [112]:
r0_rect = f"R0_rect: " + " ".join([str(ele) for ele in original_R_ref0_rect2.reshape((-1))]) + "\n"

r0_rect

'R0_rect: 0.9998817 0.01511453 -0.002841595 -0.01511724 0.9998853 -0.000933851 0.002827154 0.0009766976 0.9999955\n'

## Extract Velo to Cam Transformation

In [113]:
tr_velo_cam = f"Tr_velo_to_cam: " + " ".join([str(ele) for ele in T_velo_cam2.reshape((-1))]) + "\n"

tr_velo_cam

'Tr_velo_to_cam: 609.695406376067 -721.4215945704468 -1.2512597150460192 -78.39596581299884 180.38420147090642 7.644801418162001 -719.651521810669 -100.98428666361731 0.9999453839058352 0.00012436576507402576 0.010451302653646818 -0.26664102765164976\n'

## Extract IMU to Velo Transformation

In [114]:
tr_imu_velo = f"Tr_imu_to_velo: " + " ".join([str(ele) for ele in T_imu_velo.reshape((-1))]) + "\n"

tr_imu_velo

'Tr_imu_to_velo: 0.9999976 0.0007553071 -0.002035826 -0.8086759 -0.0007854027 0.9998898 -0.01482298 0.3195559 0.002024406 0.01482454 0.9998881 -0.7997231 0.0 0.0 0.0 1.0\n'

## Write to File

In [115]:
output = rect_matrices + r0_rect + tr_velo_cam + tr_imu_velo
print(output)

P0:  7.215377e+02 0.000000e+00 6.095593e+02 0.000000e+00 0.000000e+00 7.215377e+02 1.728540e+02 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
P1:  7.215377e+02 0.000000e+00 6.095593e+02 -3.875744e+02 0.000000e+00 7.215377e+02 1.728540e+02 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
P2:  7.215377e+02 0.000000e+00 6.095593e+02 4.485728e+01 0.000000e+00 7.215377e+02 1.728540e+02 2.163791e-01 0.000000e+00 0.000000e+00 1.000000e+00 2.745884e-03
P3:  7.215377e+02 0.000000e+00 6.095593e+02 -3.395242e+02 0.000000e+00 7.215377e+02 1.728540e+02 2.199936e+00 0.000000e+00 0.000000e+00 1.000000e+00 2.729905e-03
R0_rect: 0.9998817 0.01511453 -0.002841595 -0.01511724 0.9998853 -0.000933851 0.002827154 0.0009766976 0.9999955
Tr_velo_to_cam: 609.695406376067 -721.4215945704468 -1.2512597150460192 -78.39596581299884 180.38420147090642 7.644801418162001 -719.651521810669 -100.98428666361731 0.9999453839058352 0.00012436576507402576 0.010451302653646818 -0.266641027

## Populate the Folder

In [116]:
for i in range(len(bin_paths)):
    with open(f"{PREFIX}/new_point_clouds/training/calib/{str(i).zfill(6)}.txt", "w") as fp:
        fp.write(output)